In [9]:
import praw
import pandas as pd
import json
import datetime
from timeit import default_timer as timer
import random

#testing autocomplete
%config IPCompleter.greedy=True

In [21]:
client_id= 'UTeG611HjpoNLg' #'zB8oNVtsNw-A2A'
client_secret= 	'oVIpII6MhJb8Rsmtm1mvmhBVyIg' #'U0fn882ZjwhZhR9pejWthakz3iA'
reddit_user_agent='test code for /u/ismn5960_scraper'

reddit = praw.Reddit(client_id= client_id,
                     client_secret= client_secret,
                     user_agent='app for /u/ISMN5960_scraper')

Version 5.3.0 of praw is outdated. Version 5.4.0 was released Wednesday March 28, 2018.


##### Previously, this function was made to take a start date, an end date, and a subreddit and then pull extended data for submissions to that subreddit between the start and end date. This was done using the PRAW Python package. 

##### However around _____ day Reddit stopping using the search service that allowed to me to pull submission info from a set of dates. This required me to adjust my data collection method. 

##### Now this function is made to take file with a list of submission ids and used the PRAW Python wrapper to get extended data for each of those submissions and then put it in a separate Json file for analysis. Submission ids are gather using a third party service called Pushift.io. The methods used for gathering the submission ids can be found here: 

In [22]:
def submissionInfoScraper(file_with_ids):
    
    file = open(file_with_ids , "rb")
    jsonObject = json.load(file)
    file.close()
    
    start_timer = timer()
    current_date = datetime.date.today()
    submission_count = 0
    subreddit_submission_info = []
    fields = ('title', 'author', 'subreddit','selftext', 'num_comments', 'score', 'upvote_ratio', 'created_utc', 'gilded', 'url', 'over_18')
    
    for entry in jsonObject['id']:
        submission = reddit.submission(id=entry)
        turn_into_non_lazy_object = submission.title
        to_dict = (vars(submission))
        individual_submission_dict = {field:str(to_dict[field]) for field in fields}
        individual_submission_dict['ups'] = round(float(individual_submission_dict['score']) / float(individual_submission_dict['upvote_ratio']))
        individual_submission_dict['downs'] = (individual_submission_dict['ups'] - float(individual_submission_dict['score']))    
        subreddit_submission_info.append(individual_submission_dict)
        submission_count += 1
        
    with open(file.name.split('-')[0] + '-' + str(current_date) + '-submissionData.json', 'w', encoding='utf-8') as file:
        json.dump(subreddit_submission_info, file)
        file.close()
    
    print('/r/' + file.name.split('-')[0], 'submission count: ' + str(submission_count))
    print('File saved as: ' + file.name.split('-')[0] + '-' + str(current_date) + '-subMissiondata.json')
    print('Program took ' + str(timer() - start_timer) + ' seconds to run') #in seconds  

In [24]:
submissionInfoScraper('the_Donald-submission-ids.json')

/r/the_Donald submission count: 188795
File saved as: the_Donald-2018-04-27-subMissiondata.json
Program took 27394.644117731626 seconds to run


#### However for this project, I had to cut down on the amount of submissions I gathered so that it wouldn't take as long to gather the data. The below code was used to get the full data on 1/10th of the total submissions I had ids for. 

In [13]:
file = open('the_Donald-submission-ids.json' , "rb")
jsonObject = json.load(file)
file.close()

test = list(jsonObject['id'])

random.shuffle(test)

In [17]:
print(len(test)/10)

188795.6


In [ ]:
start_timer = timer()
current_date = datetime.date.today()
submission_count = 0
subreddit_submission_info = []
fields = ('title', 'author', 'subreddit','selftext', 'num_comments', 'score', 'upvote_ratio', 'created_utc', 'gilded', 'url', 'over_18')
    
for entry in test[:188795]:
    submission = reddit.submission(id=entry)
    turn_into_non_lazy_object = submission.title
    to_dict = (vars(submission))
    individual_submission_dict = {field:str(to_dict[field]) for field in fields}
    individual_submission_dict['ups'] = round(float(individual_submission_dict['score']) / float(individual_submission_dict['upvote_ratio']))
    individual_submission_dict['downs'] = (individual_submission_dict['ups'] - float(individual_submission_dict['score']))    
    subreddit_submission_info.append(individual_submission_dict)
    submission_count += 1
        
with open(file.name.split('-')[0] + '-' + str(current_date) + '-submissionData.json', 'w', encoding='utf-8') as file:
    json.dump(subreddit_submission_info, file)
    file.close()
    
print('/r/' + file.name.split('-')[0], 'submission count: ' + str(submission_count))
print('File saved as: ' + file.name.split('-')[0] + '-' + str(current_date) + '-subMissiondata.json')
print('Program took ' + str(timer() - start_timer) + ' seconds to run') #in seconds  